In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import __init_path__
import env

In [3]:
from pyquaternion import Quaternion
from collections import OrderedDict
from slam.linalg import (convert_euler_angles_to_rotation_matrix, 
                         form_se3, 
                         split_se3,
                         GlobalTrajectory,
                         RelativeTrajectory,
                         euler_to_quaternion,
                         shortest_path_with_normalization)

In [4]:
path_to_trajectory = '/dbstore/datasets/Odometry_team/KITTI_odometry_2012/00'
columns = ['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z']
df_gt = pd.read_csv('{}/df.csv'.format(path_to_trajectory), index_col=0)
gt_trajectory = RelativeTrajectory.from_dataframe(df_gt[['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z']]).to_global()
new_pd = gt_trajectory.to_dataframe()

In [5]:
pred_euler = pred_dist = np.array([0,0,0])
pred_components = np.array([0,0,0,0,0,0])
pred_quaternion = Quaternion(np.array([1,0,0,0]))
distances=[]
euclid_euler=[]
components=[]
abs_distance=[]
symetric_distance=[]
intrinsic_distance=[]
aprox_distance=[]
euclid_quaternion=[]
for i, values in enumerate(new_pd.values):
    distances.append( np.linalg.norm(pred_dist-values[:3])) 
    euler_x, euler_y, euler_z = shortest_path_with_normalization(pred_euler,values[3:])
    euclid_euler_distance = np.sqrt(euler_x**2 + euler_y**2 + euler_z**2)
    euclid_euler.append(euclid_euler_distance)
    components.append(pred_components-values)
    quaternion = Quaternion(euler_to_quaternion(values[3:]))
    aprox_distance.append( 1 - (quaternion.unit * pred_quaternion.unit).unit)
    abs_distance.append(Quaternion.absolute_distance(quaternion.unit ,pred_quaternion.unit ))
    intrinsic_distance.append(Quaternion.distance(quaternion.unit ,pred_quaternion.unit))
    symetric_distance.append(Quaternion.sym_distance(quaternion.unit ,pred_quaternion.unit))
    euclid_quaternion.append(min( (quaternion.unit - pred_quaternion.unit).norm,
                            (quaternion.unit + pred_quaternion.unit).norm ))
    pred_euler = values[3:]
    pred_components = values
    pred_dist = values[:3]
    pred_quaternion = quaternion